Press shift-enter to run each cell. You can edit the code in each cell, and you can also re-run each cell. The lines that start with # are comments that give you instructions on what you can edit.

In [ ]:
#this cell imports the functions needed in the notebook
from __future__ import print_function
import numpy as np
import os

from ipypathchooser import PathChooser
from ipysliceviewer import SliceViewer

import ipywidgets as widgets

%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from IPython.display import display

from reconstruction import recon_setup, recon

import pickle
recons_todo = [] #list of recon_dictionary variables so that you can accumulate a bunch of them and then run them at once.

In [ ]:
#location of your raw data on nersc
p = PathChooser("/global/cfs/cdirs/als/data_mover/8.3.2/raw/dyparkinson")

#output folder name (on NERSC scratch)
subdirectoryname = 'dulaOutput1'

outputPath = os.path.join(os.path.expandvars("$SCRATCH"),subdirectoryname)
if not os.path.exists(outputPath):
    os.mkdir(outputPath)
pickledparamsfile = f'{subdirectoryname}.pkl'

p

In [ ]:
#this sets the path and filename variables.
#we recommend you set the output path to be your scratch drive
inputPath,filename = os.path.split(p.chosen_path)

#this is where the reconstruction settings are set up.
#Open the reconstruction.py file to see comments on options.
functioninput = {
    "sinoused": (-1,1,1), #uncommenting this allows you to reconstruct just 1 slice instead of all of them
    "cor": None,# None, #set something other than None if you want to automatically detect center of rotation
    "filename": filename,
    "inputPath": inputPath,
    "outputPath": outputPath,
    "filetype": 'dxfile',
    "timepoint": 0, #don't worry too much about setting time point here, it only affects COR detection.
    "doFWringremoval": 0,
    "writenormalized": 0,
    "dorecon": 1,
    "corFunction": 'pc',
    "doPhaseRetrieval": 0,
    "alphaReg": 0.1,
    "writereconstruction": 0,
    "doOutliers1D": 1,
    "castTo8bit": 0,
    "cast8bit_min": -10,
    "cast8bit_max": 30,
    "verbose_printing": 1,
    "doPolarRing": 0,
    "Rarc": 20,
    "Rmaxwidth": 10,
    "Rtmax": 10,
    "Rthr": 10,
    "Rtmin": .2,
    "chunk_proj": 25,
    "chunk_sino": 25,
}
recon_dictionary, cor_tomo = recon_setup(**functioninput)
print(f"center of rotation: {recon_dictionary['cor']}")

In [ ]:
#manually set center of rotation based on overlay of first and last image (assumes 180 degree rotation)
if isinstance(cor_tomo,np.ndarray):  
    firstimage = np.squeeze(cor_tomo[0,:,:])
    lastimageflipped = np.squeeze(cor_tomo[1,:,::-1])
    shiftedlastimage = np.zeros_like(lastimageflipped)
    imagewidth = cor_tomo.shape[2]
    cor_shift_initial_value = 2*(recon_dictionary['cor'] - imagewidth/2)
    non = lambda s: s if s<0 else None
    mom = lambda s: max(0,s)

    fig, ax = plt.subplots(constrained_layout=True)
    fig.canvas.toolbar_position = 'right'
    fig.canvas.header_visible = False
    img = ax.imshow(firstimage-lastimageflipped,cmap='gray',vmin=-0.2,vmax=0.2)
    
    def updateimage(ox):
        recon_dictionary['cor'] = imagewidth/2.0 + ox/2.0
        shiftedlastimage[:,mom(ox):non(ox)] = lastimageflipped[:, mom(-ox):non(-ox)]
        img.set_array(firstimage - shiftedlastimage)
        ax.set_title(f"COR: {recon_dictionary['cor']}")
        fig.canvas.draw()
        fig.canvas.flush_events()
    
    slider_xshift = widgets.IntSlider(description='Shift Image', min = -300, max = 300, step=1, value = cor_shift_initial_value, layout=widgets.Layout(width='50%'))
    uishiftsliders = widgets.HBox([slider_xshift])
    outshiftsliders = widgets.interactive_output(updateimage,{'ox':slider_xshift});
    display(uishiftsliders,outshiftsliders)
else:
    print("COR was already set by user, no display available")

In [ ]:
#this cell reconsructs one slice
#(or the number of slices specified in sinoused)
print(f"COR: {recon_dictionary['cor']}")
rec, tomo = recon(**recon_dictionary)

In [ ]:
#This cell visualizes the reconstructed slice(s)
s = SliceViewer(volume=rec)
s.figure_size = [900, 900] #adjust this number to adjust the size of the figure/image
margin = s.figure_size[1] - 140
s.gridbox.layout = widgets.Layout(width='auto', height='auto', grid_gap=f'0px 10px',grid_template_columns=f'{s.figure_size[0]}px 180px', grid_template_rows=f'140px {margin}px 36px 60px', grid_template_areas=''' "figure toolbar" "figure mini-map""min-max-slider min-max-slider""bottom-bar bottom-bar"''',)
s.min_max_slider.min = -10 #if needed, adjust min and max to set brightness and contrast
s.min_max_slider.max = 30 #if needed, adjust min and max to set brightness and contrast
s

In [ ]:
#This cell takes the parameters most recently used and
#adds doing a full reconstruction to the to-do list
functioninput['sinoused'] = None
functioninput['writereconstruction'] = 1
functioninput['verbose_printing'] = 0
functioninput['cor'] = recon_dictionary['cor']
recon_dictionary, cor_tomo = recon_setup(**functioninput)

recons_todo.append(recon_dictionary)
with open(os.path.join(outputPath,pickledparamsfile), 'wb') as f:
    pickle.dump(recons_todo, f, protocol=pickle.HIGHEST_PROTOCOL)
print(f'Parameters added to list saved in {os.path.join(outputPath,pickledparamsfile)}')
print(f'Number of reconstructions in recons_todo = {len(recons_todo)}')

In [ ]:
for i in range(0,len(recons_todo)):
    recon_dictionary = recons_todo[i]
    rec, tomo = recon(**recon_dictionary)

The rest of the cells are in case you lost track of what you were doing
or had to restart the notebook and want to load a previous to do list

In [ ]:
# list the data sets on recon_todo
# for i in range(0,len(recons_todo)):
#     print(recons_todo[i]['filename'])

In [ ]:
#If a parameters file is saved previously but wasn't run,
#it can be opened here to run the reconstructions in the next cell

# with open(os.path.join(outputPath,pickledparamsfile), 'rb') as f:
#     recons_todo = pickle.load(f)